<a href="https://colab.research.google.com/github/Chancelor2023/foodrecognitionknu/blob/main/Japanese_food_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.3/896.3 kB 35.2 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# copy kaggle.json to /root/.kaggle/ folder so that kaggle cli can access it.
!mkdir /.kaggle
!mv kaggle.json /.kaggle
!mv /.kaggle /root/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d rkuo2000/uecfood256

Dataset URL: https://www.kaggle.com/datasets/rkuo2000/uecfood256
License(s): unknown
100% 3.94G/3.94G [01:55<00:00, 37.0MB/s]
100% 3.94G/3.94G [01:55<00:00, 36.6MB/s]


In [ ]:
import zipfile
with zipfile.ZipFile("uecfood256.zip", 'r') as zip_ref:
    zip_ref.extractall("food-256")

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17067, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 17067 (delta 24), reused 28 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17067/17067), 15.68 MiB | 24.40 MiB/s, done.
Resolving deltas: 100% (11714/11714), done.
/content/yolov5


In [ ]:
import os
import random
from shutil import copy2
from PIL import Image

# 1. Charger les catégories depuis category.txt
category_file = "food-256/UECFOOD256/category.txt"
with open(category_file, 'r', encoding='utf-8') as f:
    classes = [line.strip().split()[1] for line in f.readlines()[1:]]  # Ignorer la première ligne

# 2. Définir les ratios
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2
TEST_RATIO = 0.1

# 3. Créer la structure des dossiers
base_path = "food-256"
splits = ["train", "val", "test"]
for split in splits:
    os.makedirs(os.path.join(base_path, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(base_path, split, "labels"), exist_ok=True)

# 4. Parcourir les dossiers de catégories
uec_path = os.path.join(base_path, "UECFOOD256")
for category_dir in os.listdir(uec_path):
    category_path = os.path.join(uec_path, category_dir)
    if not os.path.isdir(category_path) or not category_dir.isdigit():
        continue

    bb_info_file = os.path.join(category_path, "bb_info.txt")
    if not os.path.exists(bb_info_file):
        continue

    # Charger les images
    images = [img for img in os.listdir(category_path) if img.endswith(".jpg")]
    random.shuffle(images)  # Mélanger les images

    # Répartir les images
    split_train = int(len(images) * TRAIN_RATIO)
    split_val = int(len(images) * (TRAIN_RATIO + VAL_RATIO))

    train_images = images[:split_train]
    val_images = images[split_train:split_val]
    test_images = images[split_val:]

    # Parcourir chaque image
    for image_name in images:
        # Définir le chemin source et destination
        image_path = os.path.join(category_path, image_name)
        split = (
            "train" if image_name in train_images else
            "val" if image_name in val_images else
            "test"
        )
        dest_image_dir = os.path.join(base_path, split, "images")
        dest_label_dir = os.path.join(base_path, split, "labels")
        dest_image_path = os.path.join(dest_image_dir, image_name)
        dest_label_path = os.path.join(dest_label_dir, image_name.replace(".jpg", ".txt"))

        # Charger les dimensions de l'image
        with Image.open(image_path) as img:
            image_width, image_height = img.size

        # Extraire et convertir les annotations
        with open(bb_info_file, 'r') as f_in, open(dest_label_path, 'w') as f_out:
            for line in f_in:
                parts = line.strip().split()
                if len(parts) < 5 or parts[0] != image_name[:-4]:
                    continue

                x1, y1, x2, y2 = map(int, parts[1:5])
                class_id = int(category_dir) - 1  # YOLO class IDs commencent à 0
                x_center = (x1 + x2) / (2 * image_width)
                y_center = (y1 + y2) / (2 * image_height)
                width = (x2 - x1) / image_width
                height = (y2 - y1) / image_height

                # Écrire les annotations YOLO
                f_out.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

        # Copier l'image dans le dossier correspondant
        copy2(image_path, dest_image_path)

print("Conversion terminée. Les données sont organisées.")


Conversion terminée. Les données sont organisées.


In [ ]:
import shutil

# Path to the folder you want to delete
folder_path = "/content/reduced-food-256"

# Remove the folder and all its contents
shutil.rmtree(folder_path)
print(f"Folder {folder_path} has been deleted.")


Folder /content/reduced-food-256 has been deleted.


In [ ]:
import os

# Chemins des dossiers d'images
base_path = "food-256"
train_path = os.path.join(base_path, "train", "images")
val_path = os.path.join(base_path, "val", "images")
test_path = os.path.join(base_path, "test", "images")

# Nombre de classes
nc = 256  # 256 classes dans votre dataset

# Charger les noms des classes depuis category.txt
category_file = "food-256/UECFOOD256/category.txt"
with open(category_file, 'r', encoding='utf-8') as f:
    classes = [line.strip().split()[1] for line in f.readlines()[1:]]  # Ignorer la première ligne (header)

# Créer le fichier data.yaml
yaml_content = f"""
train: {os.path.abspath(train_path)}
val: {os.path.abspath(val_path)}
test: {os.path.abspath(test_path)}  # Retirer cette ligne si vous n'avez pas de test

nc: {nc}

names:
"""

for i, class_name in enumerate(classes):
    yaml_content += f"  {i}: '{class_name}'\n"

# Écrire le fichier data.yaml
with open(os.path.join(base_path, "data.yaml"), 'w') as f:
    f.write(yaml_content)

print("Le fichier data.yaml a été généré avec succès.")


Le fichier data.yaml a été généré avec succès.


In [ ]:
import os
import random
import shutil

# Dossier original et dossier réduit
original_dataset = "/content/food-256"
reduced_dataset = "/content/food-256-reduced"

# Créer la structure de dossiers pour le dataset réduit
os.makedirs(reduced_dataset, exist_ok=True)
os.makedirs(os.path.join(reduced_dataset, "train/images"), exist_ok=True)
os.makedirs(os.path.join(reduced_dataset, "train/labels"), exist_ok=True)
os.makedirs(os.path.join(reduced_dataset, "val/images"), exist_ok=True)
os.makedirs(os.path.join(reduced_dataset, "val/labels"), exist_ok=True)
os.makedirs(os.path.join(reduced_dataset, "test/images"), exist_ok=True)
os.makedirs(os.path.join(reduced_dataset, "test/labels"), exist_ok=True)

# Proportion de réduction (ici, 25% des images)
reduction_ratio = 0.25

# Parcourir les sous-dossiers train, val, test
for split in ["train", "val", "test"]:
    images_dir = os.path.join(original_dataset, split, "images")
    labels_dir = os.path.join(original_dataset, split, "labels")

    reduced_images_dir = os.path.join(reduced_dataset, split, "images")
    reduced_labels_dir = os.path.join(reduced_dataset, split, "labels")

    # Lister toutes les images
    images = os.listdir(images_dir)
    num_images = len(images)

    # Sélectionner aléatoirement 25% des images
    selected_images = random.sample(images, int(num_images * reduction_ratio))

    # Copier les images et leurs annotations
    for image in selected_images:
        image_path = os.path.join(images_dir, image)
        label_path = os.path.join(labels_dir, image.replace(".jpg", ".txt"))

        shutil.copy(image_path, os.path.join(reduced_images_dir, image))
        shutil.copy(label_path, os.path.join(reduced_labels_dir, image.replace(".jpg", ".txt")))

print(f"Dataset réduit créé avec succès dans {reduced_dataset}")


Dataset réduit créé avec succès dans /content/food-256-reduced


In [ ]:
import os

# Chemins des dossiers d'images
base_path = "food-256-reduced"
train_path = os.path.join(base_path, "train", "images")
val_path = os.path.join(base_path, "val", "images")
test_path = os.path.join(base_path, "test", "images")

# Nombre de classes
nc = 256  # 256 classes dans votre dataset

# Charger les noms des classes depuis category.txt
category_file = "food-256/UECFOOD256/category.txt"
with open(category_file, 'r', encoding='utf-8') as f:
    classes = [line.strip().split()[1] for line in f.readlines()[1:]]  # Ignorer la première ligne (header)

# Créer le fichier data.yaml
yaml_content = f"""
train: {os.path.abspath(train_path)}
val: {os.path.abspath(val_path)}
test: {os.path.abspath(test_path)}  # Retirer cette ligne si vous n'avez pas de test

nc: {nc}

names:
"""

for i, class_name in enumerate(classes):
    yaml_content += f"  {i}: '{class_name}'\n"

# Écrire le fichier data.yaml
with open(os.path.join(base_path, "data.yaml"), 'w') as f:
    f.write(yaml_content)

print("Le fichier data.yaml a été généré avec succès.")


Le fichier data.yaml a été généré avec succès.


In [ ]:
!python yolov5/train.py --img 640 --batch 16 --epochs 50 --data /content/food-256-reduced/data.yaml --weights yolov5s.pt


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
      42/49      8.93G    0.01871    0.01215    0.06023         45        640:  76% 243/321 [02:09<00:34,  2.27it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      42/49      8.93G    0.01869    0.01215    0.06021         46        640:  76% 244/321 [02:10<00:30,  2.49it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      42/49      8.93G    0.01868    0.01215    0.06016         49        640:  76% 245/321 [02:10<00:34,  2.23it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
    